In [1]:
from PyQt6.QtWidgets import *
from PyQt6.QtCore import *
from PyQt6.QtGui import *
import sys

### Classe tableau pour la construction de courbe (copy/paste enabled)

In [2]:
class TableWidget(QTableWidget):

    def __init__(self, rows =26 , columns = 2, parent=None): #default size is without jibar
        super().__init__(rows, columns)
        

        self.setHorizontalHeaderLabels(["Product","Price"])
 

    def keyPressEvent(self, event):  # copy paste in table
        if event.key() == Qt.Key.Key_V and (event.modifiers() & Qt.KeyboardModifier.ControlModifier):
            selection = self.selectedIndexes()
            if selection:
                row_anchor = selection[0].row()
                column_anchor = selection[0].column()
                clipboard = QApplication.clipboard()

                rows = clipboard.text().split('\n')
                for indx_row, row in enumerate(rows):
                    values = row.split('\t')
                    for indx_col, value in enumerate(values):
                        item = QTableWidgetItem(value)
                        self.setItem(row_anchor + indx_row, column_anchor + indx_col, item)
            super().keyPressEvent(event)

### Combo box pour la convention BADDAys

In [3]:



class BusAdjbox(QComboBox):
    def __init__(self ):
        super(BusAdjbox,self).__init__()
        
        
        self.addItems(["Following","Modified Following","Preceding", "Modified Preceding", "Unadjusted"])
        self.setEditable(False)
        self.currentIndexChanged.connect(self.selectionchange)
        #self.setLayout(QHBoxLayout())
    def selectionchange(self,i):
        print(self.currentText())

        

### Combo box pour la convention comptage de j

In [4]:
class DCountbox(QComboBox):
    def __init__(self ):
        super(DCountbox,self).__init__()

        
        self.addItems(["SimpleDayCounter", "Thirty360 30/360 ",
                            "Actual/360", "Actual/365(Fixed)",
                           "Actuel/Actual", "Business / 252"
                           ])
        self.setEditable(False)
        self.currentIndexChanged.connect(self.selectionchange)
        self.setLayout(QHBoxLayout())


    def selectionchange(self,i):
        print(self.currentText())

        

In [5]:
from PyQt6.QtWidgets import *
from PyQt6.QtCore import *
from PyQt6.QtGui import *
import sys



class MainWindow(QMainWindow):

    def __init__(self):
        super(MainWindow, self).__init__()

        self.setWindowTitle("ZAR Vanilla Swap Pricer")
        #self.setFixedSize(QSize(600,800))

        layout = QGridLayout()
        
        # General Parameters
        layout.addWidget(QLabel("Calendar") ,0,0 )
        layout.addWidget(QLabel("South Africa") ,0,1 )
        layout.addWidget(QLabel("Fixing Days") ,1,0 )
        layout.addWidget(QSpinBox() ,1,1 )
        layout.addWidget(QLabel("Settlement Date") ,2,0 )
        layout.addWidget(QDateEdit(QDate.currentDate()) ,2,1 )
        layout.addWidget(QLabel("EOM") ,3,0 )
        layout.addWidget(QCheckBox() ,3,1 )

        update_curve_button = QPushButton("Curve Construction")
        update_curve_button.setSizePolicy(QSizePolicy.Policy.Expanding,QSizePolicy.Policy.Expanding)
        layout.addWidget(update_curve_button,0,3,2,2)


        layout.addWidget(QLabel("Nominal") ,7,1,alignment= Qt.AlignmentFlag(4) )
        layout.addWidget(QLCDNumber(4) ,7,3 )


        #Fixed leg parameters


        layout.addWidget(QLabel("Fixed") ,5,0 , alignment= Qt.AlignmentFlag(2) )
        layout.addWidget(QLabel("Leg") ,5,1 )

        layout.addWidget(QLabel("Effective Date") ,9,0 )
        layout.addWidget(QDateEdit() ,9,1)

        layout.addWidget(QLabel("Maturity") ,10,0 )
        layout.addWidget(QDateEdit() ,10,1 )

        layout.addWidget(QLabel("Fixed Rate") ,11,0 )
        layout.addWidget(QLineEdit() ,11,1 )

        layout.addWidget(QLabel("Payment Freq") ,12,0 )
        layout.addWidget(QComboBox() ,12,1 )

        layout.addWidget(QLabel("Day Count") ,13,0 )
        layout.addWidget(DCountbox() ,13,1 )

        layout.addWidget(QLabel("Bus adjustment") ,14,0 )
        layout.addWidget(BusAdjbox() ,14,1 )
        


        #Floating leg parameter

        layout.addWidget(QLabel("Floating") ,5,3 , alignment= Qt.AlignmentFlag(2) )
        layout.addWidget(QLabel("Leg") ,5,4 )

        layout.addWidget(QLabel("Effective Date") ,9,3 )
        layout.addWidget(QDateEdit() ,9,4)

        layout.addWidget(QLabel("Maturity") ,10,3)
        layout.addWidget(QDateEdit() ,10,4 )

        layout.addWidget(QLabel("Index") ,11,3 )
        layout.addWidget(QLabel("3M-Jibar") ,11,4)

        layout.addWidget(QLabel("Spread") ,12,3 )
        layout.addWidget(QLineEdit() ,12,4 )

        layout.addWidget(QLabel("Last index fix") ,13,3 )
        layout.addWidget(QLineEdit() ,13,4 )

        layout.addWidget(QLabel("Pmt Freq") ,14,3)
        layout.addWidget(QComboBox() ,14,4 )

        layout.addWidget(QLabel("Day Count") ,15,3)
        layout.addWidget(DCountbox() ,15,4 )

        layout.addWidget(QLabel("BusDay Adj") ,16,3)
        layout.addWidget(BusAdjbox() ,16,4 )


        


        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)


app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec()
        

0